In [1]:
%%writefile Book.java
public class Book {
    private int id;
    private String title;
    private String author;
    private boolean available;

    public Book(int id, String title, String author, boolean available) {
        this.id = id;
        this.title = title;
        this.author = author;
        this.available = available;
    }

    public int getId() { return id; }
    public String getTitle() { return title; }
    public String getAuthor() { return author; }
    public boolean isAvailable() { return available; }

    public void setAvailable(boolean available) {
        this.available = available;
    }

    @Override
    public String toString() {
        return id + "," + title + "," + author + "," + available;
    }

    public static Book fromString(String line) {
        String[] data = line.split(",");
        return new Book(
            Integer.parseInt(data[0]),
            data[1],
            data[2],
            Boolean.parseBoolean(data[3])
        );
    }
}


Writing Book.java


In [2]:
%%writefile Transaction.java
import java.time.LocalDate;

public class Transaction {
    private int transactionId;
    private int bookId;
    private String userName;
    private String type; // ISSUE or RETURN
    private LocalDate date;

    public Transaction(int transactionId, int bookId, String userName, String type) {
        this.transactionId = transactionId;
        this.bookId = bookId;
        this.userName = userName;
        this.type = type;
        this.date = LocalDate.now();
    }

    @Override
    public String toString() {
        return transactionId + "," + bookId + "," + userName + "," + type + "," + date;
    }
}


Writing Transaction.java


In [3]:
%%writefile Library.java
import java.io.*;
import java.util.*;

public class Library {

    private ArrayList<Book> books = new ArrayList<>();
    private static final String BOOK_FILE = "books.txt";
    private static final String TRANSACTION_FILE = "transactions.txt";

    public Library() {
        loadBooksFromFile();
    }

    public void addBook(Book book) {
        books.add(book);
        saveBooksToFile();
        System.out.println("Book added successfully.");
    }

    public void removeBook(int id) {
        Book b = getBookById(id);
        if (b != null) {
            books.remove(b);
            saveBooksToFile();
            System.out.println("Book removed.");
        } else {
            System.out.println("Book not found.");
        }
    }

    public void showAllBooks() {
        if (books.isEmpty()) {
            System.out.println("No books in library.");
            return;
        }

        for (Book b : books) {
            System.out.println(
                b.getId() + " | " + b.getTitle() + " | " + b.getAuthor() + " | " +
                (b.isAvailable() ? "Available" : "Issued")
            );
        }
    }

    public void searchBook(String keyword) {
        boolean found = false;
        for (Book b : books) {
            if (b.getTitle().toLowerCase().contains(keyword.toLowerCase()) ||
                b.getAuthor().toLowerCase().contains(keyword.toLowerCase())) {
                System.out.println(
                    b.getId() + " | " + b.getTitle() + " | " + b.getAuthor()
                );
                found = true;
            }
        }
        if (!found) System.out.println("No matching book found.");
    }

    public void issueBook(int id, String userName) {
        Book b = getBookById(id);

        if (b == null) {
            System.out.println("Book not found.");
            return;
        }

        if (!b.isAvailable()) {
            System.out.println("Book already issued.");
            return;
        }

        b.setAvailable(false);
        saveBooksToFile();
        saveTransaction(new Transaction(generateTransactionId(), id, userName, "ISSUE"));

        System.out.println("Book issued successfully.");
    }

    public void returnBook(int id, String userName) {
        Book b = getBookById(id);

        if (b == null) {
            System.out.println("Book not found.");
            return;
        }

        if (b.isAvailable()) {
            System.out.println("Book is already in library.");
            return;
        }

        b.setAvailable(true);
        saveBooksToFile();
        saveTransaction(new Transaction(generateTransactionId(), id, userName, "RETURN"));

        System.out.println("Book returned successfully.");
    }

    private int generateTransactionId() {
        return (int)(Math.random() * 100000);
    }

    private Book getBookById(int id) {
        for (Book b : books)
            if (b.getId() == id) return b;
        return null;
    }

    private void saveBooksToFile() {
        try (PrintWriter pw = new PrintWriter(new FileWriter(BOOK_FILE))) {
            for (Book b : books)
                pw.println(b);
        } catch (Exception e) {
            System.out.println("Error saving books.");
        }
    }

    private void saveTransaction(Transaction t) {
        try (PrintWriter pw = new PrintWriter(new FileWriter(TRANSACTION_FILE, true))) {
            pw.println(t);
        } catch (Exception e) {
            System.out.println("Error saving transaction.");
        }
    }

    private void loadBooksFromFile() {
        File file = new File(BOOK_FILE);
        if (!file.exists()) return;

        try (BufferedReader br = new BufferedReader(new FileReader(file))) {
            String line;
            while ((line = br.readLine()) != null) {
                books.add(Book.fromString(line));
            }
        } catch (Exception e) {
            System.out.println("Error loading books.");
        }
    }
}


Writing Library.java


In [4]:
%%writefile Main.java
import java.util.Scanner;

public class Main {
    public static void main(String[] args) {

        Library library = new Library();
        Scanner sc = new Scanner(System.in);

        while (true) {
            System.out.println("\n===== LIBRARY MANAGEMENT SYSTEM =====");
            System.out.println("1. Add Book");
            System.out.println("2. Remove Book");
            System.out.println("3. Show All Books");
            System.out.println("4. Search Book");
            System.out.println("5. Issue Book");
            System.out.println("6. Return Book");
            System.out.println("7. Exit");
            System.out.print("Enter your choice: ");

            int choice = sc.nextInt();

            switch (choice) {

                case 1:
                    System.out.print("Enter ID: ");
                    int id = sc.nextInt();
                    sc.nextLine();
                    System.out.print("Enter Title: ");
                    String title = sc.nextLine();
                    System.out.print("Enter Author: ");
                    String author = sc.nextLine();
                    library.addBook(new Book(id, title, author, true));
                    break;

                case 2:
                    System.out.print("Enter Book ID to remove: ");
                    library.removeBook(sc.nextInt());
                    break;

                case 3:
                    library.showAllBooks();
                    break;

                case 4:
                    sc.nextLine();
                    System.out.print("Enter keyword: ");
                    library.searchBook(sc.nextLine());
                    break;

                case 5:
                    System.out.print("Enter Book ID to issue: ");
                    int issueId = sc.nextInt();
                    sc.nextLine();
                    System.out.print("Enter User Name: ");
                    String u1 = sc.nextLine();
                    library.issueBook(issueId, u1);
                    break;

                case 6:
                    System.out.print("Enter Book ID to return: ");
                    int returnId = sc.nextInt();
                    sc.nextLine();
                    System.out.print("Enter User Name: ");
                    String u2 = sc.nextLine();
                    library.returnBook(returnId, u2);
                    break;

                case 7:
                    System.out.println("Exiting...");
                    return;

                default:
                    System.out.println("Invalid choice!");
            }
        }
    }
}


Writing Main.java


In [5]:
!javac *.java


In [ ]:
!java Main



===== LIBRARY MANAGEMENT SYSTEM =====
1. Add Book
2. Remove Book
3. Show All Books
4. Search Book
5. Issue Book
6. Return Book
7. Exit
Enter your choice: 2
Enter Book ID to remove: 7
Book not found.

===== LIBRARY MANAGEMENT SYSTEM =====
1. Add Book
2. Remove Book
3. Show All Books
4. Search Book
5. Issue Book
6. Return Book
7. Exit
Enter your choice: 1
Enter ID: 23
Enter Title: Hello
Enter Author: Me
Book added successfully.

===== LIBRARY MANAGEMENT SYSTEM =====
1. Add Book
2. Remove Book
3. Show All Books
4. Search Book
5. Issue Book
6. Return Book
7. Exit
Enter your choice: 4
Enter keyword: 23
No matching book found.

===== LIBRARY MANAGEMENT SYSTEM =====
1. Add Book
2. Remove Book
3. Show All Books
4. Search Book
5. Issue Book
6. Return Book
7. Exit
Enter your choice: 4
Enter keyword: hello
23 | Hello | Me

===== LIBRARY MANAGEMENT SYSTEM =====
1. Add Book
2. Remove Book
3. Show All Books
4. Search Book
5. Issue Book
6. Return Book
7. Exit
Enter your choice: 3
23 | Hello | Me | A